In [1]:
%%capture
!pip install sentence-transformers tqdm datasets

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from scipy.stats import spearmanr
import os
from tqdm import tqdm
from datasets import Dataset
from sklearn.metrics import mean_squared_error
import torch
from google.colab import drive

drive.mount('/content/drive')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda


In [3]:
data_file = '/content/drive/MyDrive/embeddings/responses.csv'
model_name = 'all-MiniLM-L6-v2'
epochs = 3
batch_size = 16
output_dir = '/content/drive/MyDrive/embeddings/fine_tuned_model'


In [4]:
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    if 'Unnamed: 0' in df.columns:
        df.rename(columns={'Unnamed: 0': 'question'}, inplace=True)
        df.to_csv(file_path, index=False)
    return df

def prepare_training_data(df):
    return [
        InputExample(texts=[str(row['question']).lower().strip(),
                            str(row[f'source{i}']).lower().strip()],
                     label=float(row[f'relevance{i}']) / 4.0)
        for _, row in df.iterrows()
        for i in range(1, 6)
        if not pd.isna(row[f'relevance{i}']) and row[f'relevance{i}'] != -1
    ]

def fine_tune_model(training_examples, model_name='all-MiniLM-L6-v2', epochs=3, batch_size=16):
    model = SentenceTransformer(model_name)
    model = model.to(device)
    train_dataloader = DataLoader(training_examples, shuffle=True, batch_size=batch_size)
    train_loss = losses.CosineSimilarityLoss(model)
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=epochs, warmup_steps=100, show_progress_bar=True)
    return model

def evaluate_model(model, test_examples):
    true_scores, pred_scores = [], []
    for example in tqdm(test_examples):
        true_score = example.label
        embeddings = model.encode(example.texts, device=device)
        pred_score = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
        true_scores.append(true_score)
        pred_scores.append(pred_score)

    true_scores, pred_scores = np.array(true_scores), np.array(pred_scores)
    correlation, p_value = spearmanr(true_scores, pred_scores)
    mse = mean_squared_error(true_scores, pred_scores)
    return correlation, mse, true_scores, pred_scores


In [5]:
df = load_and_preprocess_data(data_file)
training_examples = prepare_training_data(df)
train_examples, test_examples = train_test_split(training_examples, test_size=0.2, random_state=42)

In [6]:
model = fine_tune_model(train_examples, model_name=model_name, epochs=epochs, batch_size=batch_size)

os.makedirs(output_dir, exist_ok=True)
model.save(os.path.join(output_dir, 'fine_tuned_model'))


Step,Training Loss


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [7]:
base_model = SentenceTransformer(model_name)
fine_tuned_model = SentenceTransformer(os.path.join(output_dir, 'fine_tuned_model'))

base_corr, base_mse, base_true, base_pred = evaluate_model(base_model, test_examples)
ft_corr, ft_mse, ft_true, ft_pred = evaluate_model(fine_tuned_model, test_examples)

print(f"Base Model - Correlation: {base_corr:.4f}, MSE: {base_mse:.4f}")
print(f"Fine-tuned Model - Correlation: {ft_corr:.4f}, MSE: {ft_mse:.4f}")

# Cell 7: Qualitative Analysis
for i in range(min(5, len(test_examples))):
    example = test_examples[i]
    print(f"\nQuestion: {example.texts[0]}")
    print(f"Source: {example.texts[1][:100]}...")
    print(f"True Score: {example.label:.4f}")
    print(f"Base Model Prediction: {base_pred[i]:.4f}")
    print(f"Fine-tuned Model Prediction: {ft_pred[i]:.4f}")


100%|██████████| 535/535 [00:04<00:00, 107.34it/s]

Base Model - Correlation: 0.6581, MSE: 0.1224
Fine-tuned Model - Correlation: 0.7460, MSE: 0.0829

Question: when should a nurse contact the medical control physician regarding patients with cardiac arrhythmias
Source: 1. usually found in patients with pre-existing conduction defects. may be found subcutaneously in th...
True Score: 0.0000
Base Model Prediction: 0.4508
Fine-tuned Model Prediction: 0.2716

Question: what is the largest species of bear
Source: 1. any child with potential stroke including all bch ed and children from outside bch via critical c...
True Score: 0.0000
Base Model Prediction: -0.0177
Fine-tuned Model Prediction: -0.0005

Question: what considerations should be made for a patient undergoing postpyloric feeding tube insertion in terms of antianxiety medication comfort measures and distraction techniques
Source: pain management plan should be discussed preoperatively with surgery, neonatology and nursing. if in...
True Score: 0.2500
Base Model Prediction: 0.4161
